# Clustering Assignment: Segmenting and Clustering Neighborhoods in Toronto

## Part 1: Scraping Data from Wikipedia

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

1. Start by creating a new Notebook for this assignment.
2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

<img src="https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/7JXaz3NNEeiMwApe4i-fLg_40e690ae0e927abda2d4bde7d94ed133_Screen-Shot-2018-06-18-at-7.17.57-PM.png?expiry=1587340800000&hmac=xG2qdgypGGbgPLSFwGEvKwC0vkkrdbfoQlD5vGEsei4" align="left" width=30% height=30%>

3. To create the above dataframe:
 - The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
 - Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
 - More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods:  Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
 - If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
 - Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
 - In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
4. Submit a link to your Notebook on your Github repository. (10 marks)

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np
import requests

#!pip install lxml
import lxml
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

Libraries imported.


In [2]:
# Get data from Wikipedia using Pandas
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# Read the first table of the website
df = pd.read_html(url)[0]

# Delete rows where Borough is 'Not assigned'
df = df[df.Borough!= "Not assigned"]

# Check if there's any blank cells in Neighborhood Column
num_na_neigh = (df['Neighborhood'] == '').sum()
print(num_na_neigh)

0


There's no blank cells in Neighborhood Column

In [3]:
df.reset_index(drop=True, inplace=True)
df = df.apply(lambda x: x.str.replace(' /',','))

# define the dataframe columns
column_names = ['PostalCode','Borough', 'Neighborhood'] 

# instantiate the dataframe
df.columns = column_names

df.head()

PostalCode           Borough                                 Neighborhood
0        M3A        North York                                    Parkwoods
1        M4A        North York                             Victoria Village
2        M5A  Downtown Toronto                    Regent Park, Harbourfront
3        M6A        North York             Lawrence Manor, Lawrence Heights
4        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government

In [4]:
df.shape

(103, 3)

## Part 2: Add Latitude and Longitude Data

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. 

Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

Use the Geocoder package or the csv file to create the following dataframe:

<img src="https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/HZ3jNHNOEeiMwApe4i-fLg_f44f0f10ccfaf42fcbdba9813364e173_Screen-Shot-2018-06-18-at-7.18.16-PM.png?expiry=1587340800000&hmac=VIwNbPmfWf8g7LhteC-55cH-5ZVlflzhI6oCqNIUxzc" align="left" height=30% width=30% >

Once you are able to create the above dataframe, submit a link to the new Notebook on your Github repository. (2 marks)

In [5]:
url="https://cocl.us/Geospatial_data"
geo_data=pd.read_csv("https://cocl.us/Geospatial_data")

df2 = df.join(geo_data.set_index('Postal Code'), on='PostalCode')
df2.head()

PostalCode           Borough                                 Neighborhood  \
0        M3A        North York                                    Parkwoods   
1        M4A        North York                             Victoria Village   
2        M5A  Downtown Toronto                    Regent Park, Harbourfront   
3        M6A        North York             Lawrence Manor, Lawrence Heights   
4        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   

    Latitude  Longitude  
0  43.753259 -79.329656  
1  43.725882 -79.315572  
2  43.654260 -79.360636  
3  43.718518 -79.464763  
4  43.662301 -79.389494

## Part 3: Explore and Cluster the neighborhoods in Toronto

### Explore the neighborhoods in Toronto

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

1. to add enough Markdown cells to explain what you decided to do and to report any observations you make.
2. to generate maps to visualize your neighborhoods and how they cluster together.

Once you are happy with your analysis, submit a link to the new Notebook on your Github repository. (3 marks)

In [7]:
address = 'Toronto'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [9]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [93]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


#### Let's explore the first neighborhood in our dataframe.

In [11]:
# Get the neighborhood's name
df2.loc[0, 'Neighborhood']

'Parkwoods'

In [12]:
# Get the neighborhood's values
neighborhood_latitude = df2.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df2.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df2.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


#### Now, let's get the top 100 venues that are in Parkwoods within a radius of 500 meters.

In [13]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=XS4N4Q2KTVLOC2XR0G3ERMSBYPGYQOMU3LDUDWFK5I2NDE2B&client_secret=GLC3TSO0TYHC5EG11LRPAWK0JXVLTTCOQGHEJQGVTM5VC5ML&v=20180605&ll=43.7532586,-79.3296565&radius=500&limit=100'

In [14]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e9ac45869babe001bf0a10f'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': 

In [15]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [17]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

name         categories        lat        lng
0  Brookbanks Park               Park  43.751976 -79.332140
1  GTA Restoration    Fireworks Store  43.753396 -79.333477
2    Variety Store  Food & Drink Shop  43.751974 -79.333114

In [18]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


#### Create a function to repeat the same process to all the neighborhoods in Toronto

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
toronto_venues = getNearbyVenues(names=df2['Neighborhood'],
                                   latitudes=df2['Latitude'],
                                   longitudes=df2['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview
The Danforth West, Ri

In [22]:
print('{} venues were returned by Foursquare.'.format(toronto_venues.shape[0]))
toronto_venues.head()

2124 venues were returned by Foursquare.


Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0         Parkwoods              43.753259              -79.329656   
1         Parkwoods              43.753259              -79.329656   
2         Parkwoods              43.753259              -79.329656   
3  Victoria Village              43.725882              -79.315572   
4  Victoria Village              43.725882              -79.315572   

                    Venue  Venue Latitude  Venue Longitude     Venue Category  
0         Brookbanks Park       43.751976       -79.332140               Park  
1         GTA Restoration       43.753396       -79.333477    Fireworks Store  
2           Variety Store       43.751974       -79.333114  Food & Drink Shop  
3  Victoria Village Arena       43.723481       -79.315635       Hockey Arena  
4             Tim Hortons       43.725517       -79.313103        Coffee Shop

In [23]:
#Number of venues per neighborhood
toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Agincourt                                                               4   
Alderwood, Long Branch                                                 10   
Bathurst Manor, Wilson Heights, Downsview North                        19   
Bayview Village                                                         4   
Bedford Park, Lawrence Manor East                                      26   
Berczy Park                                                            56   
Birch Cliff, Cliffside West                                             4   
Brockton, Parkdale Village, Exhibition Place                           23   
Business reply mail Processing CentrE                                  14   
CN Tower, King and Spadina, Railway Lands, Harb...                     18   
Caledonia-Fairbanks                                                     4   
Canada Post Gateway Processing Centre                                  13   
Cedarbrae                                                               8   
Central Bay Street                                                     59   
Christie                                                               18   
Church and Wellesley                                                   73   
Clarks Corners, Tam O'Shanter, Sullivan                                15   
Cliffside, Cliffcrest, Scarborough Village West                         2   
Commerce Court, Victoria Hotel                                        100   
Davisville                                                             34   
Davisville North                                                        7   
Del Ray, Mount Dennis, Keelsdale and Silverthorn                        4   
Don Mills                                                              27   
Dorset Park, Wexford Heights, Scarborough Town ...                      5   
Downsview                                                              13   
Dufferin, Dovercourt Village                                           18   
East Toronto                                                            3   
Eringate, Bloordale Gardens, Old Burnhamthorpe,...                      8   
Fairview, Henry Farm, Oriole                                           67   
First Canadian Place, Underground city                                100   
Forest Hill North & West                                                4   
Garden District, Ryerson                                              100   
Glencairn                                                               4   
Golden Mile, Clairlea, Oakridge                                         9   
Guildwood, Morningside, West Hill                                       7   
Harbourfront East, Union Station, Toronto Islands                     100   
High Park, The Junction South                                          23   
Hillcrest Village                                                       4   
Humber Summit                                                           2   
Humberlea, Emery                                                        2   
Humewood-Cedarvale                                                      3   
India Bazaar, The Beaches West                                         20   
Kennedy Park, Ionview, East Birchmount Park                             6   
Kensington Market, Chinatown, Grange Park                              57   
Kingsview Village, St. Phillips, Martin Grove G...                      4   
Lawrence Manor, Lawrence Heights                                       15   
Lawrence Park                                                           3   
Leaside                                                                33   
Little Portugal, Trinity                                               42   
Malvern, Rouge                                                          1   
Milliken, Agincourt North, Steeles East, L'Amor.

In [24]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 268 uniques categories.


In [26]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

Yoga Studio  Accessories Store  Airport  Airport Food Court  Airport Gate  \
0            0                  0        0                   0             0   
1            0                  0        0                   0             0   
2            0                  0        0                   0             0   
3            0                  0        0                   0             0   
4            0                  0        0                   0             0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0               0                0                 0                    0   
1               0                0                 0                    0   
2               0                0                 0                    0   
3               0                0                 0                    0   
4               0                0                 0                    0   

   Antique Shop  Aquarium  Art Gallery  Arts & Crafts Store  Asian Restaurant  \
0             0         0            0                    0                 0   
1             0         0            0                    0                 0   
2             0         0            0                    0                 0   
3             0         0            0                    0                 0   
4             0         0            0                    0                 0   

   Athletics & Sports  Auto Garage  Auto Workshop  BBQ Joint  Baby Store  \
0                   0            0              0          0           0   
1                   0            0              0          0           0   
2                   0            0              0          0           0   
3                   0            0              0          0           0   
4                   0            0              0          0           0   

   Bagel Shop  Bakery  Bank  Bar  Baseball Field  Baseball Stadium  \
0           0       0     0    0               0                 0   
1           0       0     0    0               0                 0   
2           0       0     0    0               0                 0   
3           0       0     0    0               0                 0   
4           0       0     0    0               0                 0   

   Basketball Stadium  Beach  Bed & Breakfast  Beer Bar  Beer Store  \
0                   0      0                0         0           0   
1                   0      0                0         0           0   
2                   0      0                0         0           0   
3                   0      0                0         0           0   
4                   0      0                0         0           0   

   Belgian Restaurant  Bike Shop  Bistro  Boat or Ferry  Bookstore  Boutique  \
0                   0          0       0              0          0         0   
1                   0          0       0              0          0         0   
2                   0          0       0              0          0         0   
3                   0          0       0              0          0         0   
4                   0          0       0              0          0         0   

   Brazilian Restaurant  Breakfast Spot  Brewery  Bridal Shop  \
0                     0               0        0            0   
1                     0               0        0            0   
2                     0               0        0            0   
3                     0               0        0            0   
4                     0               0        0            0   

   Bubble Tea Shop  Building  Burger Joint  Burrito Place  Bus Line  \
0                0         0             0              0         0   
1                0         0             0              0         0   
2                0         0             0              0         0   
3                0         0             0              0         0   
4                0         0            

In [27]:
toronto_onehot.shape

(2124, 268)

In [29]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

Neighborhood  Yoga Studio  \
0                                        Agincourt          0.0   
1                           Alderwood, Long Branch          0.0   
2  Bathurst Manor, Wilson Heights, Downsview North          0.0   
3                                  Bayview Village          0.0   
4                Bedford Park, Lawrence Manor East          0.0   

   Accessories Store  Airport  Airport Food Court  Airport Gate  \
0                0.0      0.0                 0.0           0.0   
1                0.0      0.0                 0.0           0.0   
2                0.0      0.0                 0.0           0.0   
3                0.0      0.0                 0.0           0.0   
4                0.0      0.0                 0.0           0.0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0             0.0              0.0               0.0             0.000000   
1             0.0              0.0               0.0             0.000000   
2             0.0              0.0               0.0             0.000000   
3             0.0              0.0               0.0             0.000000   
4             0.0              0.0               0.0             0.038462   

   Antique Shop  Aquarium  Art Gallery  Arts & Crafts Store  Asian Restaurant  \
0           0.0       0.0          0.0                  0.0               0.0   
1           0.0       0.0          0.0                  0.0               0.0   
2           0.0       0.0          0.0                  0.0               0.0   
3           0.0       0.0          0.0                  0.0               0.0   
4           0.0       0.0          0.0                  0.0               0.0   

   Athletics & Sports  Auto Garage  Auto Workshop  BBQ Joint  Baby Store  \
0                 0.0          0.0            0.0        0.0         0.0   
1                 0.1          0.0            0.0        0.0         0.0   
2                 0.0          0.0            0.0        0.0         0.0   
3                 0.0          0.0            0.0        0.0         0.0   
4                 0.0          0.0            0.0        0.0         0.0   

   Bagel Shop  Bakery      Bank  Bar  Baseball Field  Baseball Stadium  \
0         0.0     0.0  0.000000  0.0             0.0               0.0   
1         0.0     0.0  0.000000  0.0             0.0               0.0   
2         0.0     0.0  0.105263  0.0             0.0               0.0   
3         0.0     0.0  0.250000  0.0             0.0               0.0   
4         0.0     0.0  0.000000  0.0             0.0               0.0   

   Basketball Stadium  Beach  Bed & Breakfast  Beer Bar  Beer Store  \
0                 0.0    0.0              0.0       0.0         0.0   
1                 0.0    0.0              0.0       0.0         0.0   
2                 0.0    0.0              0.0       0.0         0.0   
3                 0.0    0.0              0.0       0.0         0.0   
4                 0.0    0.0              0.0       0.0         0.0   

   Belgian Restaurant  Bike Shop  Bistro  Boat or Ferry  Bookstore  Boutique  \
0                 0.0        0.0     0.0            0.0        0.0       0.0   
1                 0.0        0.0     0.0            0.0        0.0       0.0   
2                 0.0        0.0     0.0            0.0        0.0       0.0   
3                 0.0        0.0     0.0            0.0        0.0       0.0   
4                 0.0        0.0     0.0            0.0        0.0       0.0   

   Brazilian Restaurant  Breakfast Spot  Brewery  Bridal Shop  \
0                   0.0            0.25      0.0     0.000000   
1                   0.0            0.00      0.0     0.000000   
2                   0.0            0.00      0.0     0.052632   
3                   0.0            0.00      0.0     0.000000   
4                   0.0            0.00      0.0     0.000000   

   Bubble Tea Shop  Building  Burger Joint  Burrito Place  Bus Line  \
0         

In [31]:
# Confirm the new size
toronto_grouped.shape

(94, 268)

In [32]:
# print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Breakfast Spot  0.25
1  Latin American Restaurant  0.25
2               Skating Rink  0.25
3                     Lounge  0.25
4                Yoga Studio  0.00


----Alderwood, Long Branch----
          venue  freq
0   Pizza Place   0.2
1      Pharmacy   0.1
2  Skating Rink   0.1
3           Gym   0.1
4           Pub   0.1


----Bathurst Manor, Wilson Heights, Downsview North----
            venue  freq
0     Coffee Shop  0.11
1            Bank  0.11
2        Pharmacy  0.05
3   Shopping Mall  0.05
4  Sandwich Place  0.05


----Bayview Village----
                 venue  freq
0                 Café  0.25
1                 Bank  0.25
2   Chinese Restaurant  0.25
3  Japanese Restaurant  0.25
4  Monument / Landmark  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0      Sandwich Place  0.08
1         Pizza Place  0.08
2          Restaurant  0.08
3    Sushi Restaurant  0.08
4  Italian Restaurant

Next, put neighborhood info into *pandas* dataframe

In [33]:
# define a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#  create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood      1st Most Common Venue  \
0                                        Agincourt  Latin American Restaurant   
1                           Alderwood, Long Branch                Pizza Place   
2  Bathurst Manor, Wilson Heights, Downsview North                       Bank   
3                                  Bayview Village                       Café   
4                Bedford Park, Lawrence Manor East           Sushi Restaurant   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0          Skating Rink        Breakfast Spot                Lounge   
1           Coffee Shop              Pharmacy                   Pub   
2           Coffee Shop              Pharmacy           Supermarket   
3    Chinese Restaurant                  Bank   Japanese Restaurant   
4        Sandwich Place    Italian Restaurant           Coffee Shop   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0   Dumpling Restaurant   Distribution Center               Dog Run   
1          Skating Rink                  Pool    Athletics & Sports   
2         Deli / Bodega      Sushi Restaurant                 Diner   
3  Ethiopian Restaurant   Empanada Restaurant     Electronics Store   
4            Restaurant           Pizza Place              Pharmacy   

         8th Most Common Venue 9th Most Common Venue   10th Most Common Venue  
0             Doner Restaurant            Donut Shop                Drugstore  
1               Sandwich Place                   Gym         Doner Restaurant  
2    Middle Eastern Restaurant            Restaurant      Fried Chicken Joint  
3  Eastern European Restaurant   Dumpling Restaurant                    Diner  
4                 Liquor Store             Locksmith  Comfort Food Restaurant

### Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [87]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 3, 1, 3, 1, 1, 1, 1, 3, 1, 4, 1, 3, 1, 1, 1, 3, 1, 1, 1, 1, 2,
       1, 3, 3, 3, 4, 3, 1, 1, 1, 1, 0, 1, 3, 1, 1, 1, 3, 1, 1, 1, 1, 1,
       3, 1, 4, 1, 1, 2, 4, 1, 0, 1, 4, 1, 3, 1, 1, 1, 3, 4, 1, 1, 1, 4,
       1, 1, 1, 3, 0, 3, 1, 1, 3, 1, 1, 1, 1, 1, 1, 4, 3, 1, 1, 1, 3, 4,
       1, 1, 1, 3, 4, 4], dtype=int32)

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [80]:
# add clustering labels
neighborhoods_venues_sorted.drop('Cluster Labels', axis=1, inplace=True)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = pd.merge(df2,neighborhoods_venues_sorted, on='Neighborhood', how='inner')

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood

#toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

PostalCode           Borough                                 Neighborhood  \
0        M3A        North York                                    Parkwoods   
1        M4A        North York                             Victoria Village   
2        M5A  Downtown Toronto                    Regent Park, Harbourfront   
3        M6A        North York             Lawrence Manor, Lawrence Heights   
4        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   

    Latitude  Longitude  Cluster Labels 1st Most Common Venue  \
0  43.753259 -79.329656               4                  Park   
1  43.725882 -79.315572               1         Grocery Store   
2  43.654260 -79.360636               1           Coffee Shop   
3  43.718518 -79.464763               1        Clothing Store   
4  43.662301 -79.389494               1           Coffee Shop   

  2nd Most Common Venue 3rd Most Common Venue  4th Most Common Venue  \
0     Food & Drink Shop       Fireworks Store                  Diner   
1           Coffee Shop            Nail Salon  Portuguese Restaurant   
2                   Pub                Bakery                   Park   
3         Women's Store           Coffee Shop             Shoe Store   
4      Sushi Restaurant                 Diner                   Park   

  5th Most Common Venue 6th Most Common Venue   7th Most Common Venue  \
0        Discount Store   Distribution Center                 Dog Run   
1          Hockey Arena         Women's Store              Donut Shop   
2        Breakfast Spot            Restaurant                 Theater   
3    Miscellaneous Shop              Boutique  Furniture / Home Store   
4                   Bar                   Spa                Beer Bar   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0      Doner Restaurant            Donut Shop              Drugstore  
1                 Diner        Discount Store    Distribution Center  
2                  Café     Health Food Store          Historic Site  
3           Event Space     Accessories Store  Vietnamese Restaurant  
4        Sandwich Place          Burger Joint          Burrito Place

Visualize the clusters

In [79]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Cluster 

In [88]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
10       North York               0   Japanese Restaurant   
30      Scarborough               0            Playground   
80  Central Toronto               0            Restaurant   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
10                Bakery                   Pub            Playground   
30         Women's Store             Drugstore                 Diner   
80            Playground            Donut Shop    Dim Sum Restaurant   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
10            Donut Shop        Discount Store   Distribution Center   
30        Discount Store   Distribution Center               Dog Run   
80                 Diner        Discount Store   Distribution Center   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
10               Dog Run      Doner Restaurant          Women's Store  
30      Doner Restaurant            Donut Shop    Dumpling Restaurant  
80               Dog Run      Doner Restaurant              Drugstore

In [89]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels      1st Most Common Venue  \
1         North York               1              Grocery Store   
2   Downtown Toronto               1                Coffee Shop   
3         North York               1             Clothing Store   
4   Downtown Toronto               1                Coffee Shop   
6         North York               1                Coffee Shop   
7         North York               1                Coffee Shop   
9   Downtown Toronto               1                Coffee Shop   
11       Scarborough               1                Golf Course   
13  Downtown Toronto               1                Coffee Shop   
14              York               1                      Field   
17      East Toronto               1                        Pub   
18  Downtown Toronto               1               Cocktail Bar   
20       Scarborough               1                Coffee Shop   
21         East York               1                Coffee Shop   
22  Downtown Toronto               1                Coffee Shop   
23  Downtown Toronto               1              Grocery Store   
25        North York               1                       Pool   
26        North York               1                       Bank   
28  Downtown Toronto               1                Coffee Shop   
31        North York               1             Clothing Store   
32        North York               1       Caribbean Restaurant   
34  Downtown Toronto               1                Coffee Shop   
35      West Toronto               1                        Bar   
36       Scarborough               1             Discount Store   
42      East Toronto               1           Greek Restaurant   
43  Downtown Toronto               1                Coffee Shop   
44      West Toronto               1                       Café   
45       Scarborough               1                     Bakery   
47      East Toronto               1       Fast Food Restaurant   
48  Downtown Toronto               1                Coffee Shop   
51       Scarborough               1        American Restaurant   
52      East Toronto               1                       Café   
53        North York               1           Sushi Restaurant   
55        North York               1               Food Service   
56       Scarborough               1                       Café   
57        North York               1           Ramen Restaurant   
58        North York               1           Ramen Restaurant   
60   Central Toronto               1                     Garden   
65   Central Toronto               1                       Park   
66   Central Toronto               1                      Trail   
67      West Toronto               1            Thai Restaurant   
69       Scarborough               1  Middle Eastern Restaurant   
70   Central Toronto               1             Clothing Store   
71   Central Toronto               1             Sandwich Place   
72      West Toronto               1                  Gift Shop   
73       Mississauga               1                      Hotel   
75       Scarborough               1  Latin American Restaurant   
76   Central Toronto               1             Sandwich Place   
77  Downtown Toronto               1                       Café   
78      West Toronto               1                Pizza Place   
81  Downtown Toronto               1                       Café   
83   Central Toronto               1                Coffee Shop   
84  Downtown Toronto               1            Airport Service   
85         Etobicoke               1                       Café   
89  Downtown Toronto               1                Coffee Shop   
92  Downtown Toronto               1                Coffee Shop   
93  Downtown Toronto               1                Coffee Shop   
95  Downtown Toronto               1                Coffee Shop   
97         Etobicoke               1                  Locksm

In [90]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
5   Scarborough               2  Fast Food Restaurant    Dim Sum Restaurant   
54         York               2  Fast Food Restaurant        Sandwich Place   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
5     Falafel Restaurant           Event Space  Ethiopian Restaurant   
54        Discount Store    Dim Sum Restaurant           Event Space   

   6th Most Common Venue 7th Most Common Venue        8th Most Common Venue  \
5    Empanada Restaurant     Electronics Store  Eastern European Restaurant   
54  Ethiopian Restaurant   Empanada Restaurant            Electronics Store   

          9th Most Common Venue 10th Most Common Venue  
5           Dumpling Restaurant              Drugstore  
54  Eastern European Restaurant    Dumpling Restaurant

In [91]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
8      East York               3           Pizza Place   
12     East York               3                  Park   
15     Etobicoke               3          Liquor Store   
16   Scarborough               3   Rental Car Location   
24   Scarborough               3   Fried Chicken Joint   
27     East York               3     Indian Restaurant   
29  West Toronto               3              Pharmacy   
37    North York               3                  Café   
38    North York               3         Grocery Store   
39    North York               3         Grocery Store   
40    North York               3         Grocery Store   
41    North York               3         Grocery Store   
50    North York               3           Pizza Place   
61          York               3         Grocery Store   
63   Scarborough               3     Indian Restaurant   
68     Etobicoke               3           Pizza Place   
74     Etobicoke               3                  Park   
79   Scarborough               3              Pharmacy   
86     Etobicoke               3         Grocery Store   
87   Scarborough               3    Chinese Restaurant   
90     Etobicoke               3           Pizza Place   
91     Etobicoke               3             Drugstore   
96  East Toronto               3  Gym / Fitness Center   

        2nd Most Common Venue      3rd Most Common Venue  \
8        Gym / Fitness Center                  Gastropub   
12               Skating Rink                        Spa   
15          Convenience Store             Cosmetics Shop   
16             Breakfast Spot         Mexican Restaurant   
24                Gas Station           Hakka Restaurant   
27                Yoga Studio  Middle Eastern Restaurant   
29                     Bakery                Pizza Place   
37         Chinese Restaurant                       Bank   
38                       Park         Athletics & Sports   
39                       Park         Athletics & Sports   
40                       Park         Athletics & Sports   
41                       Park         Athletics & Sports   
50        Empanada Restaurant              Women's Store   
61       Caribbean Restaurant                Pizza Place   
63      Vietnamese Restaurant                  Pet Store   
68  Middle Eastern Restaurant             Discount Store   
74                Pizza Place             Sandwich Place   
79                Pizza Place               Noodle House   
86        Fried Chicken Joint       Fast Food Restaurant   
87       Fast Food Restaurant                Coffee Shop   
90                Coffee Shop                   Pharmacy   
91        Rental Car Location                 Donut Shop   
96              Garden Center         Light Rail Station   

   4th Most Common Venue 5th Most Common Venue       6th Most Common Venue  \
8               Pharmacy                  Café                    Bus Line   
12              Pharmacy            Beer Store          Athletics & Sports   
15           Coffee Shop           Pizza Place                  Beer Store   
16     Electronics Store                  Bank                Intersection   
24                  Bank  Caribbean Restaurant          Athletics & Sports   
27        Sandwich Place          Burger Joint                  Restaurant   
29  Gym / Fitness Center                   Bar                 Supermarket   
37   Japanese Restaurant  Ethiopian Restaurant         Empanada Restaurant   
38            Food Truck          Liquor Store  Construction & Landscaping   
39            Food Truck          Liquor Store  Construction & Landscaping   
40            Food Truck          Liquor Store  Construction & Landscaping   
41            Food Truck          Liquor Store  Construction & Landscaping   
50            Donut Shop                 Diner              Discount Store   
61            Donut Shop                 Diner              Discount Store   
63   

In [92]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
0         North York               4                  Park   
19              York               4                  Park   
33         East York               4                  Park   
46        North York               4                  Park   
49        North York               4                  Park   
59   Central Toronto               4                  Park   
62              York               4                  Park   
64        North York               4                  Park   
82       Scarborough               4                  Park   
88  Downtown Toronto               4                  Park   
94         Etobicoke               4                  Park   

   2nd Most Common Venue       3rd Most Common Venue 4th Most Common Venue  \
0      Food & Drink Shop             Fireworks Store                 Diner   
19                  Pool               Women's Store               Airport   
33         Metro Station           Convenience Store            Donut Shop   
46             Cafeteria                 Event Space  Ethiopian Restaurant   
49                Bakery  Construction & Landscaping   Dumpling Restaurant   
59           Swim School                    Bus Line      Doner Restaurant   
62     Convenience Store                   Drugstore                 Diner   
64                  Bank                         Bar     Convenience Store   
82            Playground                  Donut Shop    Dim Sum Restaurant   
88            Playground                       Trail   Empanada Restaurant   
94                  Pool                       River      Doner Restaurant   

   5th Most Common Venue        6th Most Common Venue  \
0         Discount Store          Distribution Center   
19        Farmers Market                  Event Space   
33                 Diner               Discount Store   
46   Empanada Restaurant            Electronics Store   
49   Distribution Center                      Dog Run   
59                 Diner               Discount Store   
62        Discount Store          Distribution Center   
64             Drugstore          Distribution Center   
82                 Diner               Discount Store   
88     Electronics Store  Eastern European Restaurant   
94    Dim Sum Restaurant                        Diner   

          7th Most Common Venue 8th Most Common Venue 9th Most Common Venue  \
0                       Dog Run      Doner Restaurant            Donut Shop   
19         Ethiopian Restaurant   Empanada Restaurant     Electronics Store   
33          Distribution Center               Dog Run      Doner Restaurant   
46  Eastern European Restaurant   Dumpling Restaurant             Drugstore   
49             Doner Restaurant            Donut Shop             Drugstore   
59          Distribution Center               Dog Run            Donut Shop   
62                      Dog Run      Doner Restaurant            Donut Shop   
64                      Dog Run      Doner Restaurant            Donut Shop   
82          Distribution Center               Dog Run      Doner Restaurant   
88          Dumpling Restaurant  Ethiopian Restaurant             Drugstore   
94               Discount Store   Distribution Center               Dog Run   

         10th Most Common Venue  
0                     Drugstore  
19  Eastern European Restaurant  
33                    Drugstore  
46                   Donut Shop  
49  Eastern European Restaurant  
59           Falafel Restaurant  
62          Dumpling Restaurant  
64          Dumpling Restaurant  
82                    Drugstore  
88             Department Store  
94                   Donut Shop